In [3]:
# ===== ProDE-BT – Cell P1 (FIXED):
# Load data, filter devs with <2 bugs, tokenise, split, build loaders =====

import numpy as np
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader

from google.colab import drive
drive.mount('/content/drive')

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# -----------------------------
# 1) Load classifier_data_20
# -----------------------------
DATA_ROOT       = "/content/drive/MyDrive/deep_triage"
CLASSIFIER_FILE = f"{DATA_ROOT}/classifier_data_20.csv"
print("Classifier CSV:", CLASSIFIER_FILE)

df = pd.read_csv(CLASSIFIER_FILE)

# -----------------------------
# 2) Filter developers with < 2 bugs (avoid stratify error,
#    and also aligns with “active devs / usable profiles” idea)
# -----------------------------
df["owner"] = df["owner"].astype(str)
counts_per_owner = df["owner"].value_counts()
keep_owners = counts_per_owner[counts_per_owner >= 2].index
df = df[df["owner"].isin(keep_owners)].reset_index(drop=True)

print("Rows after removing devs with <2 bugs:", len(df))

# Re-encode developers AFTER filtering so IDs are contiguous
df["dev_id"] = df["owner"].astype("category").cat.codes
y_all = df["dev_id"].to_numpy(dtype=np.int64)
N_CLASSES = df["dev_id"].nunique()
print("Number of developers (>=2 bugs):", N_CLASSES)

# -----------------------------
# 3) Build text field
# -----------------------------
df["text"] = (
    df["issue_title"].fillna("").astype(str)
    + " "
    + df["description"].fillna("").astype(str)
)

# -----------------------------
# 4) Tokenisation with NLTK
# -----------------------------
import nltk
from nltk.tokenize import word_tokenize

# Ensure resources are available
nltk.download("punkt", quiet=True)
try:
    nltk.data.find("tokenizers/punkt_tab/english.pickle")
except LookupError:
    nltk.download("punkt_tab", quiet=True)

def tokenize(text):
    return [t.lower() for t in word_tokenize(text)]

print("Tokenising all bug reports...")
df["tokens"] = df["text"].astype(str).apply(tokenize)

# -----------------------------
# 5) Build vocabulary
# -----------------------------
from collections import Counter

MAX_VOCAB = 50000
PAD_TOKEN = "<PAD>"

word_freq = Counter()
for toks in df["tokens"]:
    word_freq.update(toks)

vocab = {PAD_TOKEN: 0}
for w, _ in word_freq.most_common(MAX_VOCAB - 1):
    if w not in vocab:
        vocab[w] = len(vocab)

VOCAB_SIZE = len(vocab)
print("Vocab size after cutoff:", VOCAB_SIZE)

# -----------------------------
# 6) Convert tokens -> ids, pad/truncate
# -----------------------------
MAX_LEN = 300

def tokens_to_ids(tokens):
    ids = [vocab.get(t, 0) for t in tokens]  # OOV -> 0 (PAD)
    if len(ids) > MAX_LEN:
        ids = ids[:MAX_LEN]
    elif len(ids) < MAX_LEN:
        ids = ids + [0] * (MAX_LEN - len(ids))
    return ids

df["token_ids"] = df["tokens"].apply(tokens_to_ids)

X_all = np.vstack(df["token_ids"].values).astype("int64")  # [N_bugs, MAX_LEN]
y_all = y_all.astype("int64")                              # [N_bugs]

print("X_all shape:", X_all.shape)
print("y_all shape:", y_all.shape)

# -----------------------------
# 7) Train / val / test split (80 / 10 / 10), stratified
# -----------------------------
from sklearn.model_selection import train_test_split

indices = np.arange(len(y_all))

# 80% train, 20% temp (val+test)
train_idx, temp_idx, y_train, y_temp = train_test_split(
    indices,
    y_all,
    test_size=0.2,
    random_state=42,
    stratify=y_all,
)

# From temp (20%), split into 10% val, 10% test
val_idx, test_idx, y_val, y_test = train_test_split(
    temp_idx,
    y_temp,
    test_size=0.7142857,   # gives overall 80/10/10
    random_state=42,
    stratify=y_temp,
)

print(f"Train bugs: {len(train_idx)}  Val bugs: {len(val_idx)}  Test bugs: {len(test_idx)}")

# -----------------------------
# 8) Dataset + DataLoaders (with global indices)
# -----------------------------
class BugDatasetWithIndex(Dataset):
    """
    Returns (tokens, label, global_index).
    global_index is used later for bug embeddings & developer profiles.
    """
    def __init__(self, X, y, indices):
        self.X = X
        self.y = y
        self.indices = np.array(indices, dtype=np.int64)

    def __len__(self):
        return len(self.indices)

    def __getitem__(self, i):
        idx = int(self.indices[i])
        x  = torch.from_numpy(self.X[idx])
        y  = torch.tensor(self.y[idx], dtype=torch.long)
        return x, y, idx

BATCH_SIZE = 256

train_ds = BugDatasetWithIndex(X_all, y_all, train_idx)
val_ds   = BugDatasetWithIndex(X_all, y_all, val_idx)
test_ds  = BugDatasetWithIndex(X_all, y_all, test_idx)
all_ds   = BugDatasetWithIndex(X_all, y_all, np.arange(len(y_all)))

train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True,  drop_last=False)
val_loader   = DataLoader(val_ds,   batch_size=BATCH_SIZE, shuffle=False, drop_last=False)
test_loader  = DataLoader(test_ds,  batch_size=BATCH_SIZE, shuffle=False, drop_last=False)
all_loader   = DataLoader(all_ds,   batch_size=BATCH_SIZE, shuffle=False, drop_last=False)

print("DataLoaders ready.")
xb, yb, idxb = next(iter(train_loader))
print("Example batch shapes -> xb:", xb.shape, " yb:", yb.shape, " idx:", idxb.shape)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Using device: cpu
Classifier CSV: /content/drive/MyDrive/deep_triage/classifier_data_20.csv


/tmp/ipython-input-448834846.py:22: DtypeWarning: Columns (3,4,5,6,7,8,9) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(CLASSIFIER_FILE)


Rows after removing devs with <2 bugs: 109979
Number of developers (>=2 bugs): 1032
Tokenising all bug reports...
Vocab size after cutoff: 50000
X_all shape: (109979, 300)
y_all shape: (109979,)
Train bugs: 87983  Val bugs: 6284  Test bugs: 15712
DataLoaders ready.
Example batch shapes -> xb: torch.Size([256, 300])  yb: torch.Size([256])  idx: torch.Size([256])


In [4]:
# ===== ProDE-BT – Cell P2 (NEW): SCL-BT-style encoder with attention =====
import torch.nn as nn
import torch.nn.functional as F

# ===== ProDE-BT – Cell P2: model definition =====

EMBED_DIM  = 128
HIDDEN_DIM = 256      # was 128
DROPOUT    = 0.3

class BugEncoder(nn.Module):
    """
    Text encoder: Embedding -> BiLSTM -> masked mean pooling -> tanh projection.
    """
    def __init__(self, vocab_size, embed_dim, hidden_dim):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim, padding_idx=0)
        self.lstm      = nn.LSTM(embed_dim, hidden_dim,
                                 batch_first=True, bidirectional=True)
        self.dropout   = nn.Dropout(DROPOUT)
        self.proj      = nn.Linear(2 * hidden_dim, hidden_dim)

    def forward(self, input_ids):
        mask = (input_ids != 0).unsqueeze(-1)        # [B, T, 1]
        x = self.embedding(input_ids)                # [B, T, E]
        x, _ = self.lstm(x)                          # [B, T, 2H]
        x = self.dropout(x)                          # regularisation
        x = x * mask
        lengths = mask.sum(dim=1).clamp(min=1)
        pooled = x.sum(dim=1) / lengths              # [B, 2H]
        z = torch.tanh(self.proj(pooled))            # [B, H]
        return z

class ProDEBTModel(nn.Module):
    """
    ProDE-BT: bug encoder + classifier.
    Stage-1: hard CE; Stage-2: profile-driven soft labels.
    """
    def __init__(self, vocab_size, embed_dim, hidden_dim, n_classes):
        super().__init__()
        self.encoder = BugEncoder(vocab_size, embed_dim, hidden_dim)
        self.fc      = nn.Linear(hidden_dim, n_classes)

    def forward(self, input_ids):
        z = self.encoder(input_ids)          # [B, H]
        logits = self.fc(z)                  # [B, C]
        return logits, z

VOCAB_SIZE = int(VOCAB_SIZE)
model = ProDEBTModel(VOCAB_SIZE, EMBED_DIM, HIDDEN_DIM, N_CLASSES).to(device)
print(model)


ProDEBTModel(
  (encoder): BugEncoder(
    (embedding): Embedding(50000, 128, padding_idx=0)
    (lstm): LSTM(128, 256, batch_first=True, bidirectional=True)
    (dropout): Dropout(p=0.3, inplace=False)
    (proj): Linear(in_features=512, out_features=256, bias=True)
  )
  (fc): Linear(in_features=256, out_features=1032, bias=True)
)


In [5]:
# ===== ProDE-BT – Cell P3: Stage-1 training with hard labels =====

EPOCHS_STAGE1 = 15
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-5)
criterion = nn.CrossEntropyLoss()

def run_epoch_ce(loader, train=True):
    if train:
        model.train()
    else:
        model.eval()

    total_loss = 0.0
    total_items = 0

    with torch.set_grad_enabled(train):
        for xb, yb, _ in loader:
            xb = xb.to(device, non_blocking=True)
            yb = yb.to(device, non_blocking=True)

            if train:
                optimizer.zero_grad()

            logits, _ = model(xb)
            loss = criterion(logits, yb)

            if train:
                loss.backward()
                optimizer.step()

            total_loss += loss.item() * xb.size(0)
            total_items += xb.size(0)

    return total_loss / max(total_items, 1)

best_val = float("inf")
best_state = None

for epoch in range(1, EPOCHS_STAGE1 + 1):
    train_loss = run_epoch_ce(train_loader, train=True)
    val_loss   = run_epoch_ce(val_loader,   train=False)

    if val_loss < best_val - 1e-3:   # small margin to avoid noise
        best_val = val_loss
        best_state = model.state_dict()

    print(f"Epoch {epoch:02d}  train_loss={train_loss:.4f}  val_loss={val_loss:.4f}")

# restore best Stage-1 model
if best_state is not None:
    model.load_state_dict(best_state)

print("Stage-1 training finished. Best val_loss =", best_val)


KeyboardInterrupt: 

In [ ]:
# ===== ProDE-BT – Cell P4 (FIXED): bug embeddings + developer profiles =====

import numpy as np
import torch

# 4.0 – basic sizes
N_BUGS   = X_all.shape[0]                 # total number of bugs
y_all_np = y_all.astype(np.int64)         # make sure it's a NumPy int64 array
HIDDEN_DIM = HIDDEN_DIM                   # comes from the model cell
n_devs  = N_CLASSES                       # number of developers (from P1)

# 4.1 – get bug-level embeddings from the trained Stage-1 model
model.eval()
bug_repr_all = np.zeros((N_BUGS, HIDDEN_DIM), dtype=np.float32)

with torch.no_grad():
    for xb, yb, idx in all_loader:
        xb  = xb.to(device, non_blocking=True)
        _, z = model(xb)                              # z: [B, H]
        idx_np = idx.cpu().numpy()
        bug_repr_all[idx_np] = z.cpu().numpy()

print("Bug representation matrix:", bug_repr_all.shape)

# 4.2 – aggregate to developer profiles (mean of their training-bug embeddings)
dev_repr   = np.zeros((n_devs, HIDDEN_DIM), dtype=np.float32)
dev_counts = np.zeros(n_devs, dtype=np.int64)

for g_idx in train_idx:
    d = int(y_all_np[g_idx])
    dev_repr[d] += bug_repr_all[g_idx]
    dev_counts[d] += 1

nonzero = dev_counts > 0
dev_repr[nonzero] /= dev_counts[nonzero][:, None]

print("Developer profile matrix:", dev_repr.shape)
print("Devs with no training bugs:", np.sum(~nonzero))

# 4.3 – L2-normalise for cosine similarity
def l2_normalise(arr, axis=-1, eps=1e-8):
    norm = np.linalg.norm(arr, axis=axis, keepdims=True)
    return arr / (norm + eps)

bug_repr_all_norm = l2_normalise(bug_repr_all)
dev_repr_norm     = l2_normalise(dev_repr)

# Store dev profiles as a torch tensor on the correct device
DEV_PROF_TORCH = torch.tensor(dev_repr_norm, dtype=torch.float32, device=device)
print("DEV_PROF_TORCH:", DEV_PROF_TORCH.shape)


In [ ]:
# ===== ProDE-BT – Cell P5: function to create soft labels per batch =====

TOP_M     = 10    # was 5 – let profiles suggest more candidates
ALPHA     = 0.35  # profile weight inside targets (0.3 → 0.35)
TEMP_SOFT = 0.7   # slightly higher temperature ⇒ smoother profile probs

def make_soft_labels(global_indices):
    """
    global_indices: LongTensor or numpy array of bug indices in this batch.
    Returns:
        soft_targets: [B, C]   (on current device)
        true_devs   : [B]      (LongTensor, on device)
    """
    if isinstance(global_indices, torch.Tensor):
        idx_np = global_indices.cpu().numpy()
    else:
        idx_np = np.asarray(global_indices, dtype=np.int64)

    B = len(idx_np)
    C = n_devs

    # Bug representations for this batch from Stage-1
    bug_vec = bug_repr_all_norm[idx_np]                      # [B, H]
    bug_vec_t = torch.tensor(bug_vec, dtype=torch.float32, device=device)  # [B, H]

    # Cosine similarity with developer profiles
    sim = torch.matmul(bug_vec_t, DEV_PROF_TORCH.t())        # [B, C]

    # Take TOP_M most similar developers
    M = min(TOP_M, C)
    top_vals, top_idx = torch.topk(sim, k=M, dim=1)          # [B, M]

    # Softmax over top-M (profile-driven probabilities)
    prob_top = torch.softmax(top_vals / TEMP_SOFT, dim=1)    # [B, M]

    # Build full [B, C] distribution
    soft = torch.zeros(B, C, device=device)
    soft.scatter_(1, top_idx, prob_top)

    # True developers (one-hot)
    true_devs = torch.tensor(y_all_np[idx_np], dtype=torch.long, device=device)
    one_hot = torch.zeros_like(soft)
    one_hot.scatter_(1, true_devs.view(-1, 1), 1.0)

    # Mix hard + profile distributions
    soft = (1.0 - ALPHA) * one_hot + ALPHA * soft
    soft = soft / soft.sum(dim=1, keepdim=True)

    return soft, true_devs


In [ ]:
# ===== ProDE-BT – Cell P6: Stage-2 fine-tuning with profile-driven soft labels =====

EPOCHS_STAGE2 = 12
BETA = 0.45            # weight of soft-label loss vs hard CE
LR_STAGE2 = 5e-4       # smaller LR for fine-tuning

optimizer_stage2 = torch.optim.Adam(model.parameters(),
                                    lr=LR_STAGE2,
                                    weight_decay=1e-5)

def eval_avg_hr_on_loader(loader):
    # quick Hit@9 average on given loader (no temperature scaling yet)
    model.eval()
    all_probs = []
    all_labels = []

    with torch.no_grad():
        for xb, yb, _ in loader:
            xb = xb.to(device, non_blocking=True)
            logits, _ = model(xb)
            probs = F.softmax(logits, dim=1)
            all_probs.append(probs.cpu().numpy())
            all_labels.append(yb.numpy())

    probs = np.vstack(all_probs)
    labels = np.concatenate(all_labels)
    TOPKS = [1, 3, 5, 7, 9]

    def hit_at_k_local(p, y, k):
        topk = np.argsort(-p, axis=1)[:, :k]
        hits = (topk == y.reshape(-1, 1)).any(axis=1)
        return float(hits.mean())

    hrs = [hit_at_k_local(probs, labels, k) for k in TOPKS]
    return np.mean(hrs)    # average HR@{1,3,5,7,9}

best_val_hr = -1.0
best_state_stage2 = None

for epoch in range(1, EPOCHS_STAGE2 + 1):
    model.train()
    total_loss = 0.0
    total_items = 0

    for xb, yb, idx_global in train_loader:
        xb = xb.to(device, non_blocking=True)
        idx_global = idx_global.to(device, non_blocking=True)

        soft_targets, true_devs = make_soft_labels(idx_global)

        optimizer_stage2.zero_grad()
        logits, _ = model(xb)
        log_probs = F.log_softmax(logits, dim=1)

        # Hard CE
        loss_hard = F.nll_loss(log_probs, true_devs)

        # Soft (profile-driven) CE
        loss_soft = -(soft_targets * log_probs).sum(dim=1).mean()

        loss = (1.0 - BETA) * loss_hard + BETA * loss_soft
        loss.backward()
        optimizer_stage2.step()

        total_loss   += loss.item() * xb.size(0)
        total_items  += xb.size(0)

    # evaluate average Hit@k on validation bugs
    val_hr = eval_avg_hr_on_loader(val_loader)
    print(f"[Stage-2] Epoch {epoch:02d}  train_loss={total_loss/total_items:.4f}  val_avgHR={val_hr:.4f}")

    # keep model with best validation average Hit@k
    if val_hr > best_val_hr + 1e-3:
        best_val_hr = val_hr
        best_state_stage2 = model.state_dict()

print("Stage-2 ProDE-BT training finished. Best val_avgHR =", best_val_hr)

if best_state_stage2 is not None:
    model.load_state_dict(best_state_stage2)


In [ ]:
# ===== ProDE-BT – Cell P7: temperature scaling on validation set =====

model.eval()

val_logits_list = []
val_labels_list = []

with torch.no_grad():
    for xb, yb, _ in val_loader:
        xb = xb.to(device, non_blocking=True)
        yb = yb.to(device, non_blocking=True)
        logits, _ = model(xb)
        val_logits_list.append(logits)
        val_labels_list.append(yb)

val_logits = torch.cat(val_logits_list, dim=0)
val_labels = torch.cat(val_labels_list, dim=0)
print("Validation logits:", val_logits.shape)

# Optimise log_temperature for NLL on validation data
log_T = torch.zeros(1, device=device, requires_grad=True)
optimizer_T = torch.optim.LBFGS([log_T], lr=0.1, max_iter=50)

def eval_T():
    optimizer_T.zero_grad()
    T = torch.exp(log_T)
    scaled = val_logits / T
    loss = F.cross_entropy(scaled, val_labels)
    loss.backward()
    return loss

optimizer_T.step(eval_T)
T_opt = torch.exp(log_T).detach()
print("Optimal temperature T:", float(T_opt))


In [ ]:
# ===== ProDE-BT – Cell P8 (NEW): profile-augmented logits + Hit@K =====

GAMMA_PROFILE = 5.0   # scale for profile-based logits; you can tune 2–10

def predict_probs(loader):
    model.eval()
    all_probs = []
    all_labels = []

    with torch.no_grad():
        for xb, yb, _ in loader:
            xb = xb.to(device, non_blocking=True)

            logits_base, z = model(xb)              # z: [B, H]

            # Profile-based scores: similarity to dev profiles
            z_norm = F.normalize(z, p=2, dim=1)     # [B, H]
            dev_norm = F.normalize(DEV_PROF_TORCH, p=2, dim=1)  # [C, H]
            profile_scores = torch.matmul(z_norm, dev_norm.t()) # [B, C]

            # Combine base logits + scaled profile scores
            logits = logits_base + GAMMA_PROFILE * profile_scores

            # Temperature scaling
            scaled = logits / T_opt
            probs  = F.softmax(scaled, dim=1)

            all_probs.append(probs.cpu().numpy())
            all_labels.append(yb.numpy())

    return np.vstack(all_probs), np.concatenate(all_labels)

probs_test, y_test_np = predict_probs(test_loader)

def hit_at_k(probs, y_true, k):
    topk = np.argsort(-probs, axis=1)[:, :k]
    hits = (topk == y_true.reshape(-1, 1)).any(axis=1)
    return float(hits.mean())

TOPKS = [1, 3, 5, 7, 9]
hrs_prode = []

print("=== ProDE-BT (profile-driven, noise-robust, calibrated) ===")
for k in TOPKS:
    hr = hit_at_k(probs_test, y_test_np, k)
    hrs_prode.append(hr)
    print(f"Top-{k} Hit Ratio: {hr:.4f}")

avg_hr_prode = float(np.mean(hrs_prode))
print(f"Average Hit Ratio ProDE-BT : {avg_hr_prode:.4f}")
